In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [2]:
dir_dataset = "C:/Users/0stix/Datasets/"

name_dataset = 'TPS_jan_2022'

df_train = pd.read_csv(dir_dataset+name_dataset+'/train.csv')
df_test = pd.read_csv(dir_dataset+name_dataset+'/test.csv')
df_sub = pd.read_csv(dir_dataset+name_dataset+'/sample_submission.csv')
len_train = len(df_train)
target='num_sold'

df_all = pd.concat([df_train,df_test],axis=0)

# df_all.head(18)
# df_all.tail()

In [3]:
from sklearn.preprocessing import OneHotEncoder
from holidays import Finland, Norway, Sweden

df_v0 = pd.DataFrame()

for attrib in ['country', 'store', 'product']:
#     df_v0 = pd.concat([df_v0, pd.get_dummies(df_all[attrib].astype('category').cat.codes, prefix=attrib)], axis=1)
    df_v0[attrib] = df_all[attrib].astype('category').cat.codes

df_v0['datetime'] = pd.to_datetime(df_all['date'])

# df_v0 = pd.concat([df_v0, pd.get_dummies(df_v0['datetime'].dt.year.astype('category').cat.codes, prefix='year')], axis=1)
df_v0 = pd.concat([df_v0, pd.get_dummies(df_v0['datetime'].dt.month.astype('category').cat.codes, prefix='month')], axis=1)
df_v0 = pd.concat([df_v0, pd.get_dummies(df_v0['datetime'].dt.isocalendar().week.astype('category').cat.codes, prefix='week')], axis=1)
# df_v0 = pd.concat([df_v0, pd.get_dummies(df_v0['datetime'].dt.day.astype('category').cat.codes, prefix='day')], axis=1)
df_v0 = pd.concat([df_v0, pd.get_dummies(df_v0['datetime'].dt.dayofweek.astype('category').cat.codes, prefix='dow')], axis=1)

df_v0['holidays'] = 0

lst_holidays = []
lst_holidays.append(Finland())
lst_holidays.append(Norway())
lst_holidays.append(Sweden())

df_v0['holidays'] = df_v0.apply(lambda row: 1 if row['datetime'] in lst_holidays[row['country']] else 0, axis=1)
df_v0.drop('datetime', axis=1, inplace=True)

df_v0[target] = df_all[target].copy()
df_train_v0 = df_v0[:len_train]
df_test_v0 = df_v0[len_train:]

X_, y_ = df_v0.drop([target], axis=1).values, df_v0[[target]].values
X_train, test_, y_train = X_[:len_train], X_[len_train:], y_[:len_train]

# df_v0.tail(5)
# df_v0.describe()
# df_v0.columns[20:]
# df_v0[df_v0.datetime==datetime.datetime(2018,12,23)]
# df_v0.head(18)

In [4]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X_t, X_v, y_t, y_v = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_t.shape,X_v.shape,y_t.shape,y_v.shape

((21038, 76), (5260, 76), (21038, 1), (5260, 1))

In [6]:
lgbm_reg = LGBMRegressor(n_estimators=100)
lgbm_reg.fit(X_t, y_t.flatten())
y_p = lgbm_reg.predict(X_v)

In [7]:
def smape(A, F):
    return 100 / len(A) * np.sum(np.abs(F - A) / (np.abs(A) + np.abs(F)))
smape_ = smape(y_v.flatten(), y_p)
print('smape: ', smape_)

smape:  4.128205710873964


In [8]:
df_sub[target] = lgbm_reg.predict(test_)
lst_gdp = [1.07,1.013,1.003]

df_sub['num_sold'] = df_sub.apply(lambda row: row['num_sold'] * lst_gdp[((int(row['row_id'])%18))//6], axis=1)

import datetime
now = datetime.datetime.now()
str_datetime = now.strftime("%y%m%d_%H%M%S")
df_sub.to_csv(dir_dataset+'submission-'+name_dataset+'-'+str_datetime+'.csv', index = 0)